In [1]:
import requests
import os
import uuid
import re
import datetime
base_url = 'http://localhost:8000/api/v0.0'


In [2]:
ngsi_msg = {
  "headers": [
    {
      "fiware-service": "tdm"
    },
    {
      "fiware-servicePath": "/cagliari/edge/meteo"
    },
    {
      "timestamp": 1531774294021
    }
  ],
  "body": {
    "attributes": [
      {
        "name": "altitude",
        "type": "float",
        "value": " "
      },
      {
        "name": "dateObserved",
        "type": "String",
        "value": "2018-07-16T20:51:33+00:00"
      },
      {
        "name": "location",
        "type": "geo:point",
        "value": "39.2479168, 9.1329701"
      },
      {
        "name": "timestamp",
        "type": "Integer",
        "value": "1531774293"
      },
      {
        "name": "windDirection",
        "type": "Float",
        "value": "174.545"
      },
      {
        "name": "windSpeed",
        "type": "Float",
        "value": "1.000"
      },
      {
        "name": "latitude",
        "type": "string",
        "value": "39.2479168"
      },
      {
        "name": "longitude",
        "type": "string",
        "value": "9.1329701"
      }
    ],
    "type": "WeatherObserved",
    "isPattern": "false",
    "id": "WeatherObserved:Edge-CFA703F4.esp8266-7806085.Davis"
  }
}




In [8]:
p = re.compile(r'(?P<Type>\w+):(?P<Edge>[a-zA-Z0-9_-]+)\.(?P<Node>[a-zA-Z0-9_-]+)\.(?P<Sensor>[a-zA-Z0-9_-]+)')
msg_type, edge, node_name, sensor_type_name = p.search(ngsi_msg['body']['id']).groups()
sensor_name = '{}.{}'.format(node_name, sensor_type_name)

In [9]:
def get_geometry(attributes):
    geom = {}
    for attr in attributes:
        if attr['name'] in {'latitude', 'longitude'}:
            geom[attr['name']] = float(attr['value'])
    return [geom['longitude'], geom['latitude']]
        

In [10]:
geometry = {"type": "Point", "coordinates": get_geometry(ngsi_msg['body']['attributes'])}

In [15]:
non_properties = {'latitude', 'longitude', 'timestamp', 'dateObserved', 'location'}
properties = [ attr['name'] for attr in ngsi_msg['body']['attributes'] if attr['name'] not in non_properties]
sensor_type = {
        "code": str(uuid.uuid4()),
        "type": msg_type,
        "name": sensor_type_name,
        "category": ["sensor"],
        "function": ["sensing"],
        "controlledProperty": properties
    }

In [16]:
requests.post(os.path.join(base_url, 'sensor_types'), json=[sensor_type])

<Response [200]>

In [17]:
stypecode = requests.get(os.path.join(base_url, 'sensor_types'), params={'name': sensor_type_name}).json()[0]['code']

In [18]:
sensors = [{
    'code': str(uuid.uuid4()),
    'stypecode': stypecode,
    'geometry': geometry,
    'nodecode': str(uuid.uuid4()),
    'name': sensor_name,
    'controlledProperty': prop}
   for prop in properties
]
sensors

[{'code': 'e9eca18e-1d7d-4f11-ac55-4e06f135811c',
  'controlledProperty': 'altitude',
  'geometry': {'coordinates': [9.1329701, 39.2479168], 'type': 'Point'},
  'name': 'esp8266-7806085.Davis',
  'nodecode': '6718dfd1-ee3e-41e3-8274-cd84145b4f2a',
  'stypecode': u'6e56152f-e61a-473f-b284-79b46faa8e83'},
 {'code': 'a809c342-3679-4b09-87f4-e1159fb41964',
  'controlledProperty': 'windDirection',
  'geometry': {'coordinates': [9.1329701, 39.2479168], 'type': 'Point'},
  'name': 'esp8266-7806085.Davis',
  'nodecode': 'c10bb2f6-aaaa-4263-87b8-311be2186690',
  'stypecode': u'6e56152f-e61a-473f-b284-79b46faa8e83'},
 {'code': 'f28317e9-96db-46a5-86ad-f4249d41dfa7',
  'controlledProperty': 'windSpeed',
  'geometry': {'coordinates': [9.1329701, 39.2479168], 'type': 'Point'},
  'name': 'esp8266-7806085.Davis',
  'nodecode': '4a9d82b2-ede5-4a04-89a0-5b4bad4eb77b',
  'stypecode': u'6e56152f-e61a-473f-b284-79b46faa8e83'}]

In [19]:
for sensor in sensors: 
    requests.post(os.path.join(base_url, 'sensors'), json=[sensor]).json()

In [20]:
measures = {}
time = None
for attr in ngsi_msg['body']['attributes']:
    name = attr['name']
    value = attr['value']
    if value is not None and str(value).strip() and \
                        name not in ('dateObserved', 'location', 'latitude', 'longitude'):
        if name == 'timestamp':
            time = datetime.datetime.fromtimestamp(float(value))
        else:             
            measures[name] = float(value)
measures         

{'windDirection': 174.545, 'windSpeed': 1.0}

In [21]:
sensorcodes = []
for prop, value in measures.items():
    sensorcode = requests.get(os.path.join(base_url, 'sensors'), params={'name': sensor_name, 'controlledProperty': prop}).json()[0]['code']
    sensorcodes.append(sensorcode)
    requests.post(os.path.join(base_url, 'measures'), json=[{'time': time.strftime('%Y-%m-%dT%H:%M:%SZ'), 'sensorcode': sensorcode, 'measure': {'value': value} }])


In [22]:
args = {
    'after': '2018-01-01T00:00:00Z',
    'before':'2019-01-01T00:00:00Z'
}
for sensorcode in sensorcodes:
    print(requests.get(os.path.join(base_url, 'sensors', sensorcode,'timeseries'), params=args).json())

{u'timebase': u'2018-01-01T00:00:00Z', u'data': [1.0], u'timedelta': [17016693.0]}
{u'timebase': u'2018-01-01T00:00:00Z', u'data': [174.545], u'timedelta': [17016693.0]}
